In [1]:
import sys, os
import pandas as pd 
import numpy as np 
import scipy as sc
import glob
import shutil
import ntpath
import datetime

In [2]:
import findspark
from pyspark.sql.functions import date_format
findspark.init()
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from pyspark.sql import functions as F

In [3]:
from pandas.plotting import scatter_matrix
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [4]:
schema=StructType([StructField("DATE",StringType(),True),
                   StructField("VIN",IntegerType(),True),
                   StructField("IIN",IntegerType(),True),
                   StructField("VOUT",IntegerType(),True),
                   StructField("VBATT",IntegerType(),True),
                   StructField("IBATT",IntegerType(),True),
                   StructField("TEMP",IntegerType(),True),
                   StructField("JP",StringType(),True),
                   StructField("CNT",IntegerType(),True),
                   StructField("STS",IntegerType(),True),
                   StructField("MIN",IntegerType(),True),
                   StructField("POWER",IntegerType(),True),
                   StructField("AUTOBCKLIGHT",IntegerType(),True),
                   StructField("CURRLIGHT",IntegerType(),True),
                   StructField("BRIGHTNESS",IntegerType(),True),
                   StructField("PAN1VOLT",IntegerType(),True),
                   StructField("PAN1CURR",IntegerType(),True),
                   StructField("BCK1VOLT",IntegerType(),True),
                   StructField("BCK1CURR",IntegerType(),True),
                   StructField("RSSI",IntegerType(),True),StructField("Batt_id",StringType(),False)])

In [5]:
spark = SparkSession \
    .builder \
    .appName("PySpark predictive maintenance") \
    .config("spark.executor.memory", "4g").config("spark.driver.memory", "4g")\
    .getOrCreate()
df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
df.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- VIN: integer (nullable = true)
 |-- IIN: integer (nullable = true)
 |-- VOUT: integer (nullable = true)
 |-- VBATT: integer (nullable = true)
 |-- IBATT: integer (nullable = true)
 |-- TEMP: integer (nullable = true)
 |-- JP: string (nullable = true)
 |-- CNT: integer (nullable = true)
 |-- STS: integer (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- POWER: integer (nullable = true)
 |-- AUTOBCKLIGHT: integer (nullable = true)
 |-- CURRLIGHT: integer (nullable = true)
 |-- BRIGHTNESS: integer (nullable = true)
 |-- PAN1VOLT: integer (nullable = true)
 |-- PAN1CURR: integer (nullable = true)
 |-- BCK1VOLT: integer (nullable = true)
 |-- BCK1CURR: integer (nullable = true)
 |-- RSSI: integer (nullable = true)
 |-- Batt_id: string (nullable = false)



## Leggere i miei CSV con Pyspark 

### 1- Giorno: 16/10/2020 

In [6]:
CSV_DF=spark.read.csv('./Days/20201102_030001.csv',header=True,sep=";",inferSchema=True)
CSV_DF=CSV_DF.withColumnRenamed("Batt_id", "host_id")
CSV_DF.show(2)
CSV_DF=CSV_DF.drop("_c0")

+---+-------------------+-----+---+-----+-----+-----+----+---+-----+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+
|_c0|               DATE|  VIN|IIN| VOUT|VBATT|IBATT|TEMP| JP|  CNT|STS|MIN|  POWER|AUTOBCKLIGHT|CURRLIGHT|BRIGHTNESS|PAN1VOLT|PAN1CURR|BCK1VOLT|BCK1CURR| RSSI|     host_id|
+---+-------------------+-----+---+-----+-----+-----+----+---+-----+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+
|  0|2020-11-01 00:00:50|29168|679|29097|29062|  708| 270|OFF|13957|  1|0.0|20575.0|        -1.0|     -1.0|      -1.0|    -1.0|    -1.0|    -1.0|    -1.0|-73.0|814U01M70006|
|  1|2020-11-01 00:01:49|29167|626|29097|29062|  687| 270|OFF|13958|  1|0.0|19965.0|        -1.0|     -1.0|      -1.0|    -1.0|    -1.0|    -1.0|    -1.0|-75.0|814U01M70006|
+---+-------------------+-----+---+-----+-----+-----+----+---+-----+---+---+-------+------------+---------+----------+--------+---

In [7]:
CSV_DF = CSV_DF.withColumn('time', F.date_format(F.col('DATE').cast('timestamp'), 'HH:mm:ss').alias('time'))
CSV_DF = CSV_DF.withColumn("date", F.col('DATE').cast('date').alias('date'))
CSV_DF.show(3)

+----------+-----+---+-----+-----+-----+----+---+-----+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|      date|  VIN|IIN| VOUT|VBATT|IBATT|TEMP| JP|  CNT|STS|MIN|  POWER|AUTOBCKLIGHT|CURRLIGHT|BRIGHTNESS|PAN1VOLT|PAN1CURR|BCK1VOLT|BCK1CURR| RSSI|     host_id|    time|
+----------+-----+---+-----+-----+-----+----+---+-----+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|2020-11-01|29168|679|29097|29062|  708| 270|OFF|13957|  1|0.0|20575.0|        -1.0|     -1.0|      -1.0|    -1.0|    -1.0|    -1.0|    -1.0|-73.0|814U01M70006|00:00:50|
|2020-11-01|29167|626|29097|29062|  687| 270|OFF|13958|  1|0.0|19965.0|        -1.0|     -1.0|      -1.0|    -1.0|    -1.0|    -1.0|    -1.0|-75.0|814U01M70006|00:01:49|
|2020-11-01|29167|650|29097|29053|  631| 270|OFF|13959|  1|0.0|18332.0|        -1.0|     -1.0|      -1.0|    -1.0|    -1.0|    -1.0|    -1.0|-73.0|814

### 2- Giorno 29/10/2020

In [8]:
CSV_DF1=spark.read.csv('./Days/20201103_030001.csv',header=True,sep=";",inferSchema=True)
CSV_DF1=CSV_DF1.withColumnRenamed("Batt_id", "host_id")
CSV_DF1=CSV_DF1.drop("_c0")

In [9]:
CSV_DF1 = CSV_DF1.withColumn('time', F.date_format(F.col('DATE').cast('timestamp'), 'HH:mm:ss').alias('time'))
CSV_DF1 = CSV_DF1.withColumn("date", F.col('DATE').cast('date').alias('date'))
CSV_DF1.show(3)

+----------+-----+---+-----+-----+-----+----+---+------+---+---+-----+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|      date|  VIN|IIN| VOUT|VBATT|IBATT|TEMP| JP|   CNT|STS|MIN|POWER|AUTOBCKLIGHT|CURRLIGHT|BRIGHTNESS|PAN1VOLT|PAN1CURR|BCK1VOLT|BCK1CURR| RSSI|     host_id|    time|
+----------+-----+---+-----+-----+-----+----+---+------+---+---+-----+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|2020-11-02|29238|644|28848|29140|  592| 285|OFF|593890|  1|  0|17250|         1.0|      1.0|       0.0| 11916.0|   510.0| 24036.0|   419.0|-87.0|814U01M70009|00:00:11|
|2020-11-02|29240|623|28848|29140|  595| 285|OFF|593891|  1|  0|17338|         1.0|      1.0|       0.0| 11916.0|   510.0| 24032.0|   419.0|  0.0|814U01M70009|00:01:10|
|2020-11-02|29240|596|28848|29137|  604| 285|OFF|593892|  1|  0|17598|         1.0|      1.0|       0.0| 11916.0|   510.0| 24032.0|   419.0|-87.0|814U01M70

### 3- Giorno 30/10/2020

In [10]:
CSV_DF2=spark.read.csv('./Days/20201104_030001.csv',header=True,sep=";",inferSchema=True)
CSV_DF2=CSV_DF2.withColumnRenamed("Batt_id", "host_id")
CSV_DF2=CSV_DF2.drop("_c0")
CSV_DF2=CSV_DF2.withColumn('time', F.date_format(F.col('DATE').cast('timestamp'), 'HH:mm:ss').alias('time'))
CSV_DF2=CSV_DF2.withColumn("date", F.col('DATE').cast('date').alias('date'))
CSV_DF2.show(3)

+----------+-----+---+-----+-----+-----+----+---+------+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|      date|  VIN|IIN| VOUT|VBATT|IBATT|TEMP| JP|   CNT|STS|MIN|  POWER|AUTOBCKLIGHT|CURRLIGHT|BRIGHTNESS|PAN1VOLT|PAN1CURR|BCK1VOLT|BCK1CURR| RSSI|     host_id|    time|
+----------+-----+---+-----+-----+-----+----+---+------+---+---+-------+------------+---------+----------+--------+--------+--------+--------+-----+------------+--------+
|2020-11-03|29237|602|28890|29135|  588| 230|OFF|595355|  1|0.0|17131.0|           1|      1.0|       0.0| 11916.0|   510.0| 24036.0|   419.0|-85.0|814U01M70009|00:00:01|
|2020-11-03|29237|636|28890|29136|  592| 235|OFF|595356|  1|0.0|17248.0|           1|      1.0|       0.0| 11916.0|   510.0| 24036.0|   419.0|-85.0|814U01M70009|00:01:00|
|2020-11-03|29238|613|28890|29138|  593| 235|OFF|595357|  1|0.0|17278.0|           1|      1.0|       0.0| 11916.0|   510.0| 24036.0|   419.0|-87

In [11]:
# CSV_DF['date']=''
# CSV_DF['time']=''
# for i in range(CSV_DF.shape[0]):
#     aus=CSV_DF['DATE'][i]
#     if type(aus) == str:
#         CSV_DF['date'][i]=datetime.strptime(aus, '%Y-%m-%d %H:%M:%S').date()
#         CSV_DF['time'][i]=datetime.strptime(aus, '%Y-%m-%d %H:%M:%S').time()
    

#### Unire i tre file .csv e rilevare il file differenza tra Vin e Vbat maggiore di 1200 mV

In [12]:
result = CSV_DF.union(CSV_DF2)

In [13]:
result=result.union(CSV_DF1)

In [14]:
print((result.count(), len(result.columns)))
# print((CSV_DF.count(), len(CSV_DF.columns)))
# print((CSV_DF2.count(), len(CSV_DF2.columns)))
# print((CSV_DF1.count()), len(CSV_DF1.columns))

(3007222, 22)


In [ ]:
result=result.withColumn("Diff_V", result["VIN"]-result["VBATT"])
result=result.withColumn("Critical",F.when(result.Diff_V>1200, "Yes").otherwise("No"))
#result=result.withColumn("Critical",F.when(result.Diff_V<1200),"Normal"))


In [ ]:
# To verify results 
#result.filter(result.Diff_V>1200).show()

In [ ]:
#Print only the Critical cases 
Critical_host=result.filter(result.Diff_V>1200)
Critical_host.toPandas().to_csv('mycsv.csv', sep=';', index=False)

In [ ]:
result.show(4)

In [15]:
result.toPandas().to_csv('Final_November1.csv', sep=';')

# Preparazione dei dati per il machine learning

In [ ]:
result.dtypes

In [ ]:
result.printSchema()

In [ ]:
# Cast column type 
result=result.drop("date")
result = result.withColumn("VIN", result["VIN"].cast(IntegerType()))
result = result.withColumn("IIN", result["IIN"].cast(IntegerType()))
result = result.withColumn("VOUT", result["VOUT"].cast(IntegerType()))
result = result.withColumn("VBATT", result["VBATT"].cast(IntegerType()))
result = result.withColumn("IBATT", result["IBATT"].cast(IntegerType()))
result = result.withColumn("TEMP", result["TEMP"].cast(IntegerType()))
result = result.withColumn("JP", result["JP"].cast(IntegerType()))
result = result.withColumn("CNT", result["CNT"].cast(IntegerType()))
result = result.withColumn("POWER", result["POWER"].cast(IntegerType()))
result = result.withColumn("BCK1CURR", result["BCK1CURR"].cast(IntegerType()))
result = result.withColumn("STS", result["STS"].cast(IntegerType()))
result = result.withColumn("MIN", result["MIN"].cast(IntegerType()))
####################################################################
result = result.withColumn("CURRLIGHT", result["CURRLIGHT"].cast(IntegerType()))
result = result.withColumn("BRIGHTNESS", result["BRIGHTNESS"].cast(IntegerType()))
result = result.withColumn("PAN1VOLT", result["PAN1VOLT"].cast(IntegerType()))
result = result.withColumn("PAN1CURR", result["PAN1CURR"].cast(IntegerType()))
result = result.withColumn("BCK1VOLT", result["BCK1VOLT"].cast(IntegerType()))
result = result.withColumn("RSSI", result["RSSI"].cast(IntegerType()))
result = result.withColumn("Diff_V", result["Diff_V"].cast(IntegerType()))
result = result.withColumn("AUTOBCKLIGHT", result["AUTOBCKLIGHT"].cast(IntegerType()))

In [ ]:
numeric_features = [t[0] for t in result.dtypes if t[1] == 'int']
result.select(numeric_features).describe().toPandas().transpose()

In [ ]:
# numeric_data = result.select(numeric_features).toPandas()
# axs = scatter_matrix(numeric_data, figsize=(8, 8));
# n = len(numeric_data.columns)
# for i in range(n):
#     v = axs[i, 0]
#     v.yaxis.label.set_rotation(0)
#     v.yaxis.label.set_ha('right')
#     v.set_yticks(())
#     h = axs[n-1, i]
#     h.xaxis.label.set_rotation(90)
#     h.set_xticks(())

In [ ]:
categoricalColumns = ['host_id', 'time']
stages = []

In [ ]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [ ]:
label_stringIdx = StringIndexer(inputCol = 'Critical', outputCol = 'label')
stages += [label_stringIdx]

numericCols = numeric_features
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
stages

## Pipeline 

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(result)

In [ ]:
cols=result.columns

In [ ]:
df = pipelineModel.transform(result)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

In [ ]:
pd.DataFrame(df.take(5), columns=df.columns)#.transpose()

In [ ]:
stages

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

In [ ]:
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

# 1- Logistic Regression Model